# 밸런스 게임 챗봇


##### 밸런스 게임 주제를 전달하고 이를 챗봇과 토론해볼 수 있도록 합니다.....(제가 할 수 있을까요...?)

1. 챗봇에게 밸런스 게임 주제를 출력하도록 합니다.
2. 밸런스 게임 주제가 선정되었으면 내가 답변을 하도록 하고 챗봇이 나의 생각을 바꾸기 위해 딴지를 걸게 합니다.

In [ ]:
# !pip install dotenv
# !pip install gtts
# !pip install speech_recognition
# !pip install openai
# !pip install pyglet
# !pip install SpeechRecognition


   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
    --------------------------------------- 0.8/32.9 MB 5.6 MB/s eta 0:00:06
   --- ------------------------------------ 2.9/32.9 MB 8.0 MB/s eta 0:00:04
   ------ --------------------------------- 5.0/32.9 MB 8.9 MB/s eta 0:00:04
   -------- ------------------------------- 7.1/32.9 MB 9.1 MB/s eta 0:00:03
   ----------- ---------------------------- 9.2/32.9 MB 9.2 MB/s eta 0:00:03
   ------------- -------------------------- 11.0/32.9 MB 9.3 MB/s eta 0:00:03
   -------------- ------------------------- 11.8/32.9 MB 8.4 MB/s eta 0:00:03
   ---------------- ----------------------- 13.4/32.9 MB 8.2 MB/s eta 0:00:03
   ----------------- ---------------------- 14.7/32.9 MB 8.0 MB/s eta 0:00:03
   ------------------- -------------------- 16.0/32.9 MB 7.7 MB/s eta 0:00:03
   --------------------- ------------------ 17.3/32.9 MB 7.6 MB/s eta 0:00:03
   ---------------------- ----------------- 18.9/32.9 MB 7.7 MB/s eta 0:00:02

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
from gtts import gTTS
import speech_recognition as sr

1. 주제를 만드는 함수

In [53]:
from openai import OpenAI

def make_thema(query, temperature=0.3):
    client = OpenAI()

    system_instruction = '''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다. 
    술자리에서 다같이 재미있게 얘기할 만한 연애 밸런스 게임 주제 세 개를 선정해주세요.

    ### 지시사항 ###
    - 토론이 가능한 주제를 선정할 것
    - 성인이니까 선정적인 주제도 하나 정도는 출력할 것
    - 선택하기 어렵고 애매한 주제를 선택할 것
    - 여사친/남사친 문제를 한 개 이상 포함할 것

    ### 출력 형식 ###
    🤖좋아요🤖 재미있는 주제를 세 가지 소개해드릴게요.

    주제 1:  선택지 vs 선택지.
    주제 2: 선택지 vs 선택지.
    '''
    user_message = f'''
    {query}
    '''

    response = client.chat.completions.create(

        model='gpt-4o',
        messages=[{
            'role': 'system',
            'content': [{
                'type': 'text',
                'text': system_instruction
            }]

        },
        {
            'role': 'user',
            'content': [{
                'type': 'text',
                'text': user_message

            }]

            
        }],
        response_format={
            'type':'text'
        },
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

##### 2. 챗봇 메세지를 읽어주는 함수

In [32]:
from gtts import gTTS
import pyglet
import time
from nltk.tokenize import word_tokenize, sent_tokenize

def read_chatbot(chatbot_message):      # stream 처리된 메세지
    tts_data = gTTS(
            text=chatbot_message,
            lang='ko'
        )
    tts_data.save('tts_file.mp3')

    # 저장한 음성 파일을 재생
    music = pyglet.media.load('tts_file.mp3',streaming=False)
    music.play()
    print('🤖 : ',end='')
    for sentences in chatbot_message.split(sep='.'):
        for word in word_tokenize(sentences):
            print(word,end=' ')
            time.sleep(.1)
        print()
    

##### 3. 내가 하는 말을 듣고 출력하는 함수

In [5]:
def listen_chatbot():                                                                      
    recognizer = sr.Recognizer()
    while True:
        with sr.Microphone() as source:
            print('👩‍🦰 : ',end='')
            audio = recognizer.listen(source)
            command = recognizer.recognize_google(audio, language='ko')
            if command is not None:
                
                print(command)
                print()
                
                break    
            
    return command

##### 4. chatlog를 수정하는 함수

In [54]:
def modity_chatlog(chat_log, new_message):
    if len(chat_log)%2 ==0:
        chat_log.append({ 'role': 'asistant', 'content': [{ 'type': 'text', 'text': new_message }] })
    else:
        chat_log.append({ 'role': 'user', 'content': [{ 'type': 'text', 'text': new_message }]  })
    return chat_log

In [14]:
# 제일 먼저 챗봇이 출력할 멘트 & 음성
first_input= '안녕하세요! 어떤 것을 도와드릴까요?'
command = read_chatbot(first_input)
thema_output = make_thema(command)
read_chatbot(thema_output)
listen_chatbot()

🤖 안녕하세요! 어떤 것을 도와드릴까요?🤖 안녕하세요! 어떤 것을 도와드릴까요?🤖 안녕하세요! 어떤 것을 도와드릴까요?🤖 안녕하

Exception in thread Thread-33:
Traceback (most recent call last):
  File "c:\Users\brain\anaconda3\envs\llm_env\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\brain\anaconda3\envs\llm_env\Lib\site-packages\pyglet\media\player_worker_thread.py", line 66, in run
    player.work()
  File "c:\Users\brain\anaconda3\envs\llm_env\Lib\site-packages\pyglet\media\drivers\xaudio2\adaptation.py", line 250, in work
    self._update_play_cursor()
  File "c:\Users\brain\anaconda3\envs\llm_env\Lib\site-packages\pyglet\media\drivers\xaudio2\adaptation.py", line 242, in _update_play_cursor
    self.source.audio_format.bytes_per_frame
    ^^^^^^^^^^^^^^^^^^^^^^^^
ReferenceError: weakly-referenced object no longer exists


KeyboardInterrupt: 

In [46]:
thema_output= make_thema('재미있는 밸런스 게임 주제 추천해줘!')

In [47]:
read_chatbot(thema_output)

🤖 : 🤖좋아요🤖 재미있는 주제를 세 가지 소개해드릴게요 
주제 1 : 평생 여사친/남사친과 단둘이 여행 가기 vs 평생 여사친/남사친과 단둘이 술 마시기 
주제 2 : 연인의 과거 연애사를 전부 알기 vs 연인의 과거 연애사를 전혀 모른 채로 살기 
주제 3 : 연인과 매일 밤 뜨거운 밤 보내기 vs 연인과 매일 아침 달콤한 아침 맞이하기 
어떤 것에 대해 이야기해볼까요 ? 


In [ ]:
# # 챗봇이 메세지를 순차적으로 출력하게끔 함 (stream=True 이용)

# thema_output_messages = []
# for contents in thema_output:
#     content = contents.choices[0].delta.content

#     if content is not None:
#         thema_output_messages.append(content)
#         print(thema_output_messages[-1], end="")



    주제 1. 평생 여사친/남사친과 단둘이 여행 금지 vs 평생 연인과 단둘이 여행 금지

    주제 2. 연인이 과거에 유명한 연애 리얼리티 프로그램에 출연한 적 있음 vs 연인이 과거에 성인 잡지 모델로 활동한 적 있음

    주제 3. 연인이 매일 전 애인과 연락하는 것 허용 vs 연인이 매주 전 애인과 만나는 것 허용

    어떤 것에 대해 이야기해볼까요?

In [55]:
new_system_instruction = f'''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다.

    {thema_output.split()[1:]}

    어떤 것에 대해서 이야기하고 싶은지 먼저 물어보고, 상대가 선정한 주제에 대해 토론해봅시다.
    상대가 생각을 바꿀 수 있도록 영악하고 악독하게 비꼬면서 토론을 해주세요.

    
    ### 토론시 지시사항 ###
    - 3문장 이내로 반말로 대답할 것
    - 이해하는 척 하면서 교묘하게 상대가 선택한 선택지에서 악화될 상황을 가정해 말할 것
    - 만약 상대가 설득당했다면 "ㅋㅋㅋㅋ 재밌었다 그치?" 를 그대로 출력할 것
    - 예시를 적극 활용할 것

    ### 출력 형식 ###
    아하 그렇구나. 근데 그렇게 했다가 애인이 남사친/여사친과 눈 맞으면 어떡해?

    '''

In [52]:
from openai import OpenAI

def choose_n_discuss(chat_log, temperature=0.3):
    client = OpenAI()
    response = client.chat.completions.create(

        model='gpt-4o',
        messages= chat_log,
        response_format={
            'type':'text'
        },
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [ ]:
chat_log = [{ 'role': 'system', 'content': [{ 'type': 'text', 'text': new_system_instruction }] }

            ]

NameError: name 'user_message' is not defined